11/28/2025 - Nafisa - Load all raw data files
**Purpose:**  
Load all CSV, JSON, and Parquet files from the fall2025_L directory, merge them into a single DataFrame, clean labels, and save the combined dataset for downstream processing.


**Interpretation / Findings:** 
- Total 5 unique labels found: BENIGN, DoS Hulk, DoS GoldenEye, DoS Slowhttptest, Heartbleed.
- Heartbleed (11 rows) removed as required.
- Created binary target column Attack → 1 for attacks, 0 for benign.
- Final dataset size after cleaning:
    -  Attack = 1: 56,112 rows
    - Attack = 0: 5,005 rows
- Output saved to: data/interm/combined_raw.csv

**Notes for Team:**
The combined dataset is now ready for feature engineering and model training. Please ensure to review the cleaned labels and the binary target column for consistency before proceeding. 

### Done with Task